<a href="https://colab.research.google.com/github/Papa-Panda/Paper_reading/blob/main/llama_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install -U langchain-community

In [4]:
from huggingface_hub import notebook_login
notebook_login()

# # use a bert model to do sentiment analysis


In [12]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# model_name = "distilbert-base-uncased-finetuned-sst-2-english"
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# text = 'This is a test sentence'
# inputs = tokenizer(text, return_tensors='pt')
# outputs = model(**inputs)
# classifier = pipeline('sentiment-analysis', model=model_name)
# classifier(text)

# chat with a (small) foundational model

In [2]:
# Import necessary libraries
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import torch

In [18]:
import time

In [4]:
# # Load the LLaMA 2 model and tokenizer from Hugging Face
# model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
# # https://stackoverflow.com/questions/75725818/loading-hugging-face-model-is-taking-too-much-memory
# model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True, low_cpu_mem_usage=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
# https://stackoverflow.com/questions/76446228/setting-padding-token-as-eos-token-when-using-datacollatorforlanguagemodeling-fr
tokenizer.pad_token = tokenizer.eos_token  # Set the padding token to eos_token

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install te

In [ ]:
print("Chat with LLaMA. Type 'exit' to end the conversation.")
conversation_history = ""
conversation_ids = None  # Track tokenized conversation

while True:
    begin = time.time()
    # Get user input
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break

    # # Append the user input to the conversation history
    # conversation_history += f"User: {user_input}\nAssistant:"

    # # Tokenize and generate the response
    # inputs = tokenizer(conversation_history, return_tensors="pt", padding=True)#.to("cuda")
    # inputs["attention_mask"] = inputs["input_ids"] != tokenizer.pad_token_id  # Create the attention mask
    # outputs = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=100)

    # # Decode and display the assistant's response
    # response = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Assistant:")[-1].strip()
    # print(f"Assistant: {response}")

    # # Update conversation history
    # conversation_history += f" {response}\n"

    ## optimized verseion
    # Append user input directly into conversation
    new_input = f"User: {user_input}\nAssistant:"

    # Tokenize only the new input instead of the whole history
    new_input_ids = tokenizer(new_input, return_tensors="pt", padding=True, add_special_tokens=False).input_ids#.to("cuda")

    # Concatenate the new input with the previous conversation tokens
    if conversation_ids is None:
        conversation_ids = new_input_ids
    else:
        conversation_ids = torch.cat([conversation_ids, new_input_ids], dim=-1)

    # Generate the response
    outputs = model.generate(
        conversation_ids,
        attention_mask=conversation_ids != tokenizer.pad_token_id,
        max_new_tokens=100,
        min_new_tokens=20,  # Batch generate for better efficiency
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and display the assistant's response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Assistant:")[-1].strip()
    print(f"Assistant: {response}")

    # Update conversation history with the assistant's response
    new_response_ids = tokenizer(f" {response}\n", return_tensors="pt", add_special_tokens=False).input_ids#.to("cuda")
    conversation_ids = torch.cat([conversation_ids, new_response_ids], dim=-1)

    print('It takes ' , round(time.time() - begin), ' seconds.')


Chat with LLaMA. Type 'exit' to end the conversation.
You: hello
Assistant: Your new password is hello
It takes  184  seconds.


# # using langchanin to create an agent

In [14]:
# from transformers import pipeline
# from transformers import AutoModelForCausalLM, pipeline
# from langchain.llms import HuggingFacePipeline
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain

# # Set up a pipeline with Hugging Face
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=512)

# # Wrap the pipeline in LangChain's HuggingFacePipeline LLM
# llm = HuggingFacePipeline(pipeline=pipe)

# # Create an LLM chain with the prompt template and LLaMA-2 model
# llm_chain = LLMChain(prompt=prompt_template, llm=llm)

# # Test the chain with an example text
# input_text = "The quick brown fox jumps over the lazy dog."
# output = llm_chain.run(text=input_text)

# # Display the output
# print("Translation:", output)
